# Métodos Numéricos 

**Nombre:** Luis Alexander Lema Delgado
**Materia:** Métodos Numéricos  
**Fecha:** 12/05/2025

### IEEE 754 32 Bits

In [1]:
def decimal_a_ieee_simple(num):
    """
    Devuelve una cadena de 32 caracteres ('0' y '1')
    que representa el número 'num' en formato IEEE-754 (simple precisión).
    No se usan librerías externas, todo es “a mano”.
    """

    # 1) Bit de signo 
    bit_signo = 0 if num >= 0 else 1
    num = abs(num)                   # a partir de aquí trabajamos con valor positivo

    # 2) Separar parte entera y fraccionaria 
    parte_entera = int(num)
    parte_frac   = num - parte_entera

    # 3) Parte entera a binario (divisiones sucesivas) 
    if parte_entera == 0:
        bin_entera = '0'
    else:
        bin_entera = ''
        temp = parte_entera
        while temp > 0:
            bin_entera = str(temp % 2) + bin_entera
            temp //= 2

    # 4) Parte fraccionaria a binario (multiplicaciones sucesivas) 
    bin_frac = ''
    pasos = 0
    while parte_frac != 0 and pasos < 23:   # 23 es el máximo que interesa para la mantisa
        parte_frac *= 2
        bit = int(parte_frac)
        bin_frac += str(bit)
        parte_frac -= bit
        pasos += 1

    # 5) Unir ambas partes y localizar el “punto binario” 
    secuencia_total = bin_entera + bin_frac         # sin el punto, solo la secuencia
    exponente_val   = len(bin_entera) - 1           # desplazamiento verdadero

    # 6) Exponente con sesgo (bias = 127) 
    exp_biased = exponente_val + 127                # sesgo de IEEE-754
    # conversión manual a 8 bits
    exp_bits = ''
    temp = exp_biased
    for _ in range(8):
        exp_bits = str(temp % 2) + exp_bits
        temp //= 2

    # 7) Mantisa: todo lo que queda después del primer '1' 
    mantisa = (secuencia_total[1:] + '0' * 23)[:23]  # se rellena con ceros si hace falta

    # 8) Empaquetar resultado final 
    resultado_ieee = str(bit_signo) + exp_bits + mantisa
    return resultado_ieee

# Prueba rápida
if __name__ == "__main__":
    numero = 3.14159
    ieee = decimal_a_ieee_simple(numero)
    print("Número original :", numero)
    print("IEEE-754 (binary):", ieee)

Número original : 3.14159
IEEE-754 (binary): 01000000010010010000111111001111


### Bin integrada nativamente

In [2]:

numero = 1000
binario = bin(numero)
print("Número decimal:", numero)
print("En binario:", binario)


Número decimal: 1000
En binario: 0b1111101000


### IEEE 754 a Decimal

In [3]:

def binario_a_entero(cadena):
    """Convierte un numero binario como '101' a su valor entero (manual)."""
    resultado = 0
    potencia = 0
    for i in range(len(cadena) - 1, -1, -1):
        if cadena[i] == '1':
            resultado += 2 ** potencia
        potencia += 1
    return resultado

def ieee_a_decimal(cadena_bits):
    # Bit de signo
    signo_bit = int(cadena_bits[0])

    # Cálculo manual del exponente (8 bits después del bit de signo)
    bits_exponente = cadena_bits[1:9]
    valor_exponente = binario_a_entero(bits_exponente) - 127  # sesgo 127

    # Se forma la mantisa con el 1 implícito al inicio
    parte_mantisa = '1' + cadena_bits[9:]  # se asume número normalizado

    # Cálculo del valor decimal usando potencias de 2
    resultado = 0
    for i in range(len(parte_mantisa)):
        bit = int(parte_mantisa[i])
        potencia = valor_exponente - i
        resultado += bit * (2 ** potencia)

    # Si el bit de signo es 1, el número es negativo
    if signo_bit == 1:
        resultado *= -1

    return resultado

# Ejemplo de uso
if __name__ == "__main__":
    bits_ejemplo = '01000000010010010000111111011011'  # aproximación a 3.14159
    decimal = ieee_a_decimal(bits_ejemplo)
    print("Cadena IEEE:", bits_ejemplo)
    print("Valor decimal aproximado:", decimal)

Cadena IEEE: 01000000010010010000111111011011
Valor decimal aproximado: 3.1415927410125732


### Error por punto flotante

In [4]:

# Suma de dos valores decimales que deberían dar 0.3
valor_1 = 0.1
valor_2 = 0.2
suma = valor_1 + valor_2

# Valor teórico con el que comparamos
valor_esperado = 0.3

# Mostramos los valores obtenidos
print("Resultado de 0.1 + 0.2 =", suma)
print("Valor esperado (0.3)  =", valor_esperado)

# Comprobación directa de igualdad
if suma == valor_esperado:
    print("¿Son exactamente iguales? Sí")
else:
    print("¿Son exactamente iguales? No")

# Cálculo manual de diferencia absoluta (sin usar abs)
diferencia = suma - valor_esperado
if diferencia < 0:
    diferencia = -diferencia

print("Diferencia absoluta entre ambos valores:", diferencia)


Resultado de 0.1 + 0.2 = 0.30000000000000004
Valor esperado (0.3)  = 0.3
¿Son exactamente iguales? No
Diferencia absoluta entre ambos valores: 5.551115123125783e-17
